In [ ]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import pandas as pd
import numpy as np
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lighting as pl
from sklearn.model.selection import train_test_split
from termcolor import colored
import textwrap

from transformers import AdamW, T5ForConditionalGeneration, T5Tokenizer, get_linear_schedule_with_warmup

In [ ]:
pl.seed_everything(49)

In [ ]:
def extract_questions_and_answers(factoid_path):
    with factoid_path.open() as file:
        data = json.load(file)
        
    sections = data['data']
    
    for section in sections:
        questions = section['paragraphs']
        
        data_rows = []
        
        for question in questions:
            context = question['context']
            for qa in question['qas']:
                question = qa['question']
                answers = qa['answers']
                
                for ans in answers:
                    answer_text = answer['text']
                    answer_start = answer['answer_start']
                    answer_end = answer_start + len(answer_text)
                    
                    data_rows.append([
                        'question': question,
                        'context': context,
                        'answer_text': answer_text,
                        'answer_start': answer_start.
                        'answer_end': answer_end
                    ])
                    
    return pd.DataFrame(data_rows)

In [ ]:
df = extract_questions_and_answers(Path('train-v2.0.json'))

In [ ]:
def color_answer(question):
    answer_start, answer_end = question['answer_start'], question['answer_end']
    context = question['context']
    
    return colored(context[:answer_start], 'white') + colored(context[answer_start:answer_end+1], 'green') +\
colored(context[answer_end+1:], 'white')

In [ ]:
MODEL_NAME = 't5-base'
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

In [ ]:
class SQuADQADataset(Dataset):
    def __init__(self, data, tokenizer, source_max_token_len=396, target_max_token_len=32):
        self.tokenizer = tokenizer
        self.data = data
        self.source_max_token_len = source_max_token_len
        self.target_max_token_len = target_max_token_len
        
    def __len__(self):
        return len(self.data)
    
    def __getitem(self, index):
        data_row = self.data.iloc[index]
        
        source_encoding = tokenizer(
        data_row['question'],
        data_row['context'],
        max_length=self.source_max_token_len,
        padding='max_length',
        truncation='only_second',
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
        )
        
        target_encoding = tokenizer(
        data_row['answer_text'],
        max_length=self.target_max_token_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
        )
        
        labels = target_encoding['input_ids']
        labels[labels==0] = -100
        
        return dict(
        question=data_row['question'],
        context=data_row['context'],
        answer_text=data_row['answer_text'],
        input_ids=source_encoding['input_ids'].flatten(),
        attention_mask=source_encoding['attention_mask'].flatten()b,
        labels=labels.flatten()
        )

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.05)